In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tsfresh 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

In [159]:
training_data= pd.read_csv("./instance_training_datasets/1000_samples_desriptive_plus_target_training_1.csv")
test_data = pd.read_csv("./instance_training_datasets/1000_samples_desriptive_plus_target_test_1.csv")

In [164]:
### Model 1:
y_training_model1_ = training_data.target_task.values
X_training_model1_ = training_data.drop(['dataset_python', 'indeks_dataset_python', 'cluster', 'cell', 'target_task'], axis=1).values


In [165]:
y_test = test_data.target_task.values
X_test = test_data.drop(['dataset_python', 'indeks_dataset_python', 'cluster', 'cell', 'target_task'], axis=1).values

In [166]:
training_data

,"values__cwt_coefficients__coeff_5__w_10__widths_(2, 5, 10, 20)",values__symmetry_looking__r_0.6000000000000001,values__fft_coefficient__attr_.real.__coeff_5,values__fft_coefficient__attr_.real.__coeff_0,"values__cwt_coefficients__coeff_14__w_2__widths_(2, 5, 10, 20)",values__variance,values__augmented_dickey_fuller__attr_.usedlag.__autolag_.AIC.,values__agg_linear_trend__attr_.rvalue.__chunk_len_5__f_agg_.var.,values__change_quantiles__f_agg_.var.__isabs_True__qh_0.6__ql_0.2,values__symmetry_looking__r_0.8,...,values__symmetry_looking__r_0.55,values__change_quantiles__f_agg_.mean.__isabs_True__qh_0.6__ql_0.2,values__energy_ratio_by_chunks__num_segments_10__segment_focus_4,"values__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","values__cwt_coefficients__coeff_13__w_5__widths_(2, 5, 10, 20)",dataset_python,indeks_dataset_python,cluster,cell,target_task
0,0.216499,1.0,0.567495,3.000000e-08,0.000007,0.989583,3.0,0.462459,0.000000,1.0,...,1.0,0.000000,0.003360,0.093338,0.030488,ElectricDevices,12303,0,58,1
1,0.368806,1.0,-4.712266,-5.050000e-08,0.000011,0.996825,16.0,0.419624,0.001876,1.0,...,1.0,0.024038,0.006044,-0.139373,0.052863,UWaveGestureLibraryX,504,0,71,0
2,-0.682054,1.0,-3.507607,7.530000e-07,-0.000021,0.996825,4.0,0.580949,0.000030,1.0,...,1.0,0.002822,0.051636,0.144949,-0.096953,UWaveGestureLibraryZ,1431,0,201,1
3,-0.169362,1.0,-4.421329,-6.110000e-07,0.007360,0.996825,2.0,0.562523,0.000178,1.0,...,1.0,0.007821,0.073231,0.160020,0.026769,UWaveGestureLibraryX,2186,0,202,0
4,2.889474,1.0,45.455613,3.379000e-07,-0.062653,0.995726,12.0,0.047798,0.000481,1.0,...,1.0,0.021113,0.016015,1.514553,-0.241952,Wine,90,0,212,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4845,-0.653117,1.0,-21.771037,-1.536000e-07,-0.369600,0.996667,13.0,-0.252043,0.030332,1.0,...,1.0,0.173729,0.262861,0.833037,-1.091997,CricketY,499,4,1902,1
4846,-1.584110,1.0,1.056977,5.300000e-08,0.075737,0.996667,14.0,-0.136245,0.006290,1.0,...,1.0,0.078739,0.151435,-3.536155,0.144559,CricketX,126,4,1910,1
4847,-0.899691,1.0,19.912795,-5.070000e-07,-0.048044,0.996667,15.0,-0.148934,0.003012,1.0,...,1.0,0.049519,0.248748,-2.093441,-0.013470,CricketZ,448,4,1911,0
4848,-2.047887,1.0,13.107214,-5.382000e-07,-0.005331,0.996667,16.0,0.115400,0.013083,1.0,...,1.0,0.066290,0.151131,-1.538422,-0.157985,CricketZ,669,4,1912,0


In [167]:
cluster_groups_train = training_data.groupby(['cluster']).groups
cluster_groups_test = test_data.groupby(['cluster']).groups

In [168]:
X_training_model1_.shape

(4850, 324)

In [169]:
def build_model(X_training_model1_, y_training_model1_, fold_split=5):
    five_fold = StratifiedKFold(fold_split)
    f1_scores_folds = []
    conf_matrix_folds = []
    model1 = RandomForestClassifier(100, max_depth=None, min_samples_split=2, max_features="log2")
    
#     model1 = LogisticRegression(C=10)
    
    cnt = 0
    for train_indecies, test_indecies in five_fold.split(X_training_model1_,y_training_model1_ ):
        print("Training Fold {}".format(cnt+1))
        X_train_model1 = X_training_model1_[train_indecies, :]
        X_test_model1 = X_training_model1_[test_indecies, :]

        y_train_model1 = y_training_model1_[train_indecies]
        y_test_model1 = y_training_model1_[test_indecies]

        model1.fit(X_train_model1, y_train_model1)
        preds1 = model1.predict(X_test_model1)

        f1_scores_folds.append(f1_score(y_test_model1, preds1))
        conf_matrix_folds.append(confusion_matrix(y_test_model1, preds1))
        cnt+=1
        print("The f1 score is {}".format(f1_score(y_test_model1, preds1).round(3)))
        print("------"*10)
        
    model1.fit(X_training_model1_, y_training_model1_)
    
    return f1_scores_folds, conf_matrix_folds, model1

def predict_test(model, X_test, y_test):
    preds = model.predict(X_test)
    res = f1_score(y_test, preds)
    print("TEST F1 {} ".format(res))
    return preds, res

In [170]:
##### Model 1
model1_f1_scores, model1_confusion_matrix, model1_model  = build_model(X_training_model1_, y_training_model1_, 5)
print("MEAN F1 {} and std {}".format(np.mean(model1_f1_scores).round(3), np.std(model1_f1_scores).round(3)))
model1_preds, model1_res =  predict_test(model1_model, X_test, y_test)

Training Fold 1
The f1 score is 0.115
------------------------------------------------------------
Training Fold 2
The f1 score is 0.428
------------------------------------------------------------
Training Fold 3
The f1 score is 0.039
------------------------------------------------------------
Training Fold 4
The f1 score is 0.236
------------------------------------------------------------
Training Fold 5
The f1 score is 0.375
------------------------------------------------------------
MEAN F1 0.239 and std 0.148
TEST F1 0.24580193045087928 


In [171]:
##### Model 2

model2_f1_scores, model1_confusion_matrix, model2_model  = build_model(X_training_model1_[cluster_groups_train[0]], y_training_model1_[cluster_groups_train[0]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model2_f1_scores).round(3), np.std(model2_f1_scores).round(3)))
model2_preds, model2_res =  predict_test(model2_model, X_test, y_test)

Training Fold 1
The f1 score is 0.375
------------------------------------------------------------
Training Fold 2
The f1 score is 0.053
------------------------------------------------------------
Training Fold 3
The f1 score is 0.1
------------------------------------------------------------
Training Fold 4
The f1 score is 0.386
------------------------------------------------------------
Training Fold 5
The f1 score is 0.108
------------------------------------------------------------
MEAN F1 0.204 and std 0.145
TEST F1 0.2325234097513723 


In [172]:
##### Model 3

model3_f1_scores, model1_confusion_matrix, model3_model  = build_model(X_training_model1_[cluster_groups_train[1]], y_training_model1_[cluster_groups_train[1]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model3_f1_scores).round(3), np.std(model3_f1_scores).round(3)))
model3_preds, model3_res =  predict_test(model3_model, X_test, y_test)

Training Fold 1
The f1 score is 0.27
------------------------------------------------------------
Training Fold 2
The f1 score is 0.278
------------------------------------------------------------
Training Fold 3
The f1 score is 0.298
------------------------------------------------------------
Training Fold 4
The f1 score is 0.258
------------------------------------------------------------
Training Fold 5
The f1 score is 0.312
------------------------------------------------------------
MEAN F1 0.283 and std 0.019
TEST F1 0.08224939637043382 


In [173]:
##### Model 4

model4_f1_scores, model4_confusion_matrix, model4_model  = build_model(X_training_model1_[cluster_groups_train[2]], y_training_model1_[cluster_groups_train[2]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model4_f1_scores).round(3), np.std(model4_f1_scores).round(3)))
model4_preds, model4_res =  predict_test(model4_model, X_test, y_test)

Training Fold 1
The f1 score is 0.333
------------------------------------------------------------
Training Fold 2
The f1 score is 0.414
------------------------------------------------------------
Training Fold 3
The f1 score is 0.125
------------------------------------------------------------
Training Fold 4
The f1 score is 0.118
------------------------------------------------------------
Training Fold 5
The f1 score is 0.273
------------------------------------------------------------
MEAN F1 0.253 and std 0.116
TEST F1 0.3406473499402709 


In [174]:
##### Model 5

model5_f1_scores, model5_confusion_matrix, model5_model  = build_model(X_training_model1_[cluster_groups_train[3]], y_training_model1_[cluster_groups_train[3]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model5_f1_scores).round(3), np.std(model5_f1_scores).round(3)))
model5_preds, model5_res =  predict_test(model5_model, X_test, y_test)

Training Fold 1
The f1 score is 0.045
------------------------------------------------------------
Training Fold 2
The f1 score is 0.26
------------------------------------------------------------
Training Fold 3
The f1 score is 0.091
------------------------------------------------------------
Training Fold 4
The f1 score is 0.085
------------------------------------------------------------
Training Fold 5
The f1 score is 0.122
------------------------------------------------------------
MEAN F1 0.121 and std 0.074
TEST F1 0.1932241671372106 


In [175]:
##### Model 5

model6_f1_scores, model6_confusion_matrix, model6_model  = build_model(X_training_model1_[cluster_groups_train[4]], y_training_model1_[cluster_groups_train[4]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model6_f1_scores).round(3), np.std(model6_f1_scores).round(3)))
model6_preds, model6_res =  predict_test(model6_model, X_test, y_test)

Training Fold 1
The f1 score is 0.107
------------------------------------------------------------
Training Fold 2
The f1 score is 0.197
------------------------------------------------------------
Training Fold 3
The f1 score is 0.611
------------------------------------------------------------
Training Fold 4
The f1 score is 0.356
------------------------------------------------------------
Training Fold 5
The f1 score is 0.458
------------------------------------------------------------
MEAN F1 0.346 and std 0.18
TEST F1 0.20230377227633203 


In [176]:
cluster_groups_test[0]

Int64Index([   964,    965,    966,    967,    968,    969,    970,    971,
               972,    973,
            ...
            167057, 167059, 167060, 167062, 167063, 167064, 167065, 167066,
            167067, 167068],
           dtype='int64', length=43617)

In [177]:
cluster_groups_test[1]

Int64Index([   312,    429,    826,    986,   1196,   1311,   1455,   1492,
              1498,   1502,
            ...
            163324, 163325, 163358, 163417, 163456, 163497, 163680, 163739,
            163776, 163819],
           dtype='int64', length=23018)

In [178]:
diff_f1_scores = np.ones(5)*model1_res - np.array([model2_res, model3_res, model4_res, model5_res, model6_res])

In [179]:
output_stats = pd.DataFrame(pd.value_counts(test_data.cluster).iloc[:-1, ])
output_stats.columns = ["support"]
output_stats["abs difference test (F1 score) aggregated"] = diff_f1_scores[output_stats.index]
output_stats["f1_all_clusters"] = np.ones(5)*model1_res

output_stats["f1_individual"] = np.array([model2_res, model3_res, model4_res, model5_res, model6_res])[output_stats.index]

output_stats["train_all"] = np.array([
    f1_score(model1_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]), 
    f1_score(model1_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]), 
    f1_score(model1_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]), 
    f1_score(model1_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]), 
    f1_score(model1_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]), 
])[output_stats.index]

output_stats["train_cluster_0"] = np.array([
    f1_score(model2_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]), 
    f1_score(model2_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]), 
    f1_score(model2_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]), 
    f1_score(model2_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]), 
    f1_score(model2_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]), 
])[output_stats.index]

output_stats["train_cluster_1"] = np.array([
    f1_score(model3_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]), 
    f1_score(model3_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]), 
    f1_score(model3_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]), 
    f1_score(model3_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]), 
    f1_score(model3_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]), 
])[output_stats.index]

output_stats["train_cluster_2"] = np.array([
    f1_score(model4_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]), 
    f1_score(model4_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]), 
    f1_score(model4_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]), 
    f1_score(model4_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]), 
    f1_score(model4_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]), 
])[output_stats.index]

output_stats["train_cluster_3"] = np.array([
    f1_score(model5_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]), 
    f1_score(model5_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]), 
    f1_score(model5_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]), 
    f1_score(model5_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]), 
    f1_score(model5_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]), 
])[output_stats.index]

output_stats["train_cluster_4"] = np.array([
    f1_score(model6_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]), 
    f1_score(model6_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]), 
    f1_score(model6_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]), 
    f1_score(model6_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]), 
    f1_score(model6_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]), 
])[output_stats.index]


In [180]:
output_stats.sort_index()

,support,abs difference test (F1 score) aggregated,f1_all_clusters,f1_individual,train_all,train_cluster_0,train_cluster_1,train_cluster_2,train_cluster_3,train_cluster_4
0,43617,0.013279,0.245802,0.232523,0.279224,0.290070,0.030992,0.319010,0.183809,0.232834
1,23018,0.163553,0.245802,0.082249,0.257600,0.419422,0.263251,0.375400,0.347784,0.116107
2,1230,-0.094845,0.245802,0.340647,0.237288,0.263447,0.259615,0.317460,0.233684,0.034783
3,67843,0.052578,0.245802,0.193224,0.153029,0.051149,0.043171,0.316743,0.155318,0.044357
4,31360,0.043498,0.245802,0.202304,0.341746,0.192054,0.037160,0.396461,0.110919,0.369829


In [181]:
output_stats.sort_index().to_csv("./1000_run1.csv")

In [182]:
samples_1000_r1 = pd.read_csv("./1000_run1.csv")
samples_1000_r2 = pd.read_csv("./1000_run2.csv")
samples_1000_r3 = pd.read_csv("./1000_run3.csv")

In [184]:
(samples_1000_r1 + samples_1000_r2 + samples_1000_r1)/3

,Unnamed: 0,support,abs difference test (F1 score) aggregated,f1_all_clusters,f1_individual,train_all,train_cluster_0,train_cluster_1,train_cluster_2,train_cluster_3,train_cluster_4
0,0.0,43617.0,0.025761,0.249706,0.223945,0.284681,0.293876,0.035131,0.318466,0.144251,0.230773
1,1.0,23018.0,0.166265,0.249706,0.083442,0.271902,0.312768,0.275412,0.376764,0.283161,0.118851
2,2.0,1230.0,-0.092619,0.249706,0.342325,0.320616,0.244419,0.205109,0.385770,0.262860,0.071358
3,3.0,67843.0,0.090859,0.249706,0.158847,0.132453,0.044340,0.032614,0.321003,0.137357,0.051953
4,4.0,31360.0,0.042565,0.249706,0.207141,0.360363,0.241268,0.039592,0.397000,0.085420,0.388362


In [157]:
samples_2000_r1 = pd.read_csv("./2000_run1.csv")
samples_2000_r2 = pd.read_csv("./2000_run2.csv")
samples_2000_r3 = pd.read_csv("./2000_run3.csv")
    

In [185]:
(samples_2000_r1 + samples_2000_r2 + samples_2000_r1)/3

,Unnamed: 0,support,abs difference test (F1 score) aggregated,f1_all_clusters,f1_individual,train_all,train_cluster_0,train_cluster_1,train_cluster_2,train_cluster_3,train_cluster_4
0,0.0,42767.0,0.034658,0.252312,0.217654,0.274427,0.286560,0.083617,0.275180,0.112939,0.226157
1,1.0,22018.0,0.141315,0.252312,0.110997,0.258031,0.371875,0.253749,0.259397,0.217119,0.067894
2,2.0,230.0,-0.020314,0.252312,0.272626,0.470899,0.235031,0.224309,0.568279,0.300885,0.196402
3,3.0,66843.0,0.117819,0.252312,0.134493,0.150183,0.052105,0.087380,0.226677,0.154112,0.035544
4,4.0,30360.0,0.053350,0.252312,0.198962,0.369847,0.210674,0.055781,0.343621,0.042839,0.398174


In [80]:
output_stats.sort_index()

,support,abs difference test (F1 score) aggregated,f1_all_clusters,f1_individual,train_all,train_cluster_0,train_cluster_1,train_cluster_2,train_cluster_3,train_cluster_4
0,42767,0.017770,0.258999,0.241229,0.286581,0.284893,0.090461,0.185814,0.104081,0.264584
1,22018,0.162146,0.258999,0.096853,0.294020,0.396076,0.286866,0.032736,0.087807,0.082672
2,230,0.124994,0.258999,0.134005,0.412698,0.020408,0.222222,0.521739,0.371681,0.411429
3,66843,0.168029,0.258999,0.090970,0.097474,0.038445,0.014064,0.038814,0.100509,0.050843
4,30360,0.027016,0.258999,0.231983,0.411453,0.286358,0.070198,0.236040,0.060713,0.443179


In [ ]:
print("Train cluster 0, f1 grouped by cluster 0 ", f1_score(model2_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 0, f1 grouped by cluster 1 ", f1_score(model2_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 0, f1 grouped by cluster 2 ", f1_score(model2_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 0, f1 grouped by cluster 3 ", f1_score(model2_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 0, f1 grouped by cluster 4 ", f1_score(model2_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

In [ ]:
print("Train cluster 1, f1 grouped by cluster 0 ",f1_score(model3_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 1, f1 grouped by cluster 1 ",f1_score(model3_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 1, f1 grouped by cluster 2 ",f1_score(model3_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 1, f1 grouped by cluster 3 ",f1_score(model3_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 1, f1 grouped by cluster 4 ",f1_score(model3_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))


In [ ]:
print("Train cluster 2, f1 grouped by cluster 0 ",f1_score(model4_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 2, f1 grouped by cluster 1 ",f1_score(model4_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 2, f1 grouped by cluster 2 ",f1_score(model4_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 2, f1 grouped by cluster 3 ",f1_score(model4_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 2, f1 grouped by cluster 4 ",f1_score(model4_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

In [ ]:
print("Train cluster 3, f1 grouped by cluster 0 ",f1_score(model5_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 3, f1 grouped by cluster 1 ",f1_score(model5_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 3, f1 grouped by cluster 2 ",f1_score(model5_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 3, f1 grouped by cluster 3 ",f1_score(model5_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 3, f1 grouped by cluster 4 ",f1_score(model5_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

In [ ]:
print("Train cluster 4, f1 grouped by cluster 0 ",f1_score(model6_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 4, f1 grouped by cluster 1 ",f1_score(model6_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 4, f1 grouped by cluster 2 ",f1_score(model6_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 4, f1 grouped by cluster 3 ",f1_score(model6_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 4, f1 grouped by cluster 4 ",f1_score(model6_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

In [ ]:
print("Train all clusters, f1 grouped by cluster 0 ",f1_score(model1_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train all clusters, f1 grouped by cluster 1 ",f1_score(model2_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train all clusters, f1 grouped by cluster 2 ",f1_score(model3_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train all clusters, f1 grouped by cluster 3 ",f1_score(model4_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train all clusters, f1 grouped by cluster 4 ",f1_score(model5_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))